In [1]:
import h2o

In [2]:
import os
os.getcwd()

'/home/jovyan/work'

In [3]:
h2o.init(ip='h2o-service')

Checking whether there is an H2O instance running at http://h2o-service:54321 . connected.


H2O cluster uptime:,27 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.1
H2O cluster version age:,2 days
H2O cluster name:,root
H2O cluster total nodes:,1
H2O cluster free memory:,7.111 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [7]:
from h2o.automl import H2OAutoML
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")
# Identify the response and set of predictors
y = "response"
x = list(train.columns)  #if x is defined as all columns except the response, then x is not required
x.remove(y)
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [11]:
# Run AutoML for 30 seconds
aml = H2OAutoML(max_runtime_secs = 20*60)
aml.train(x = x, y = y, training_frame = train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [12]:
# Print Leaderboard (ranked by xval metrics)
aml.leaderboard

model_id,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20190718_144534,0.787747,0.554425,0.32943,0.43346,0.187888
XGBoost_grid_1_AutoML_20190718_144534_model_6,0.786006,0.554368,0.3342,0.433653,0.188055
StackedEnsemble_AllModels_AutoML_20190718_143928,0.785222,0.557378,0.339847,0.434925,0.18916
XGBoost_3_AutoML_20190718_144534,0.784789,0.557521,0.326983,0.434797,0.189048
XGBoost_2_AutoML_20190718_144534,0.784486,0.55508,0.330655,0.434253,0.188576
XGBoost_grid_1_AutoML_20190718_144534_model_8,0.783486,0.558377,0.321395,0.435329,0.189511
XGBoost_grid_1_AutoML_20190718_144534_model_2,0.78346,0.555465,0.324938,0.434858,0.189101
XGBoost_1_AutoML_20190718_144534,0.783373,0.556405,0.324015,0.434767,0.189023
XGBoost_grid_1_AutoML_20190718_144534_model_7,0.783342,0.558329,0.329663,0.435327,0.189509
XGBoost_1_AutoML_20190718_143928,0.78319,0.556179,0.350288,0.434876,0.189117


In [13]:
# (Optional) Evaluate performance on a test set
perf = aml.leader.model_performance(test)
perf.auc()

0.790224147431334